In [1]:
import pandas as pd
essays = pd.read_csv('C:/Users/Aileen/Downloads/opendata_essays000.gz', escapechar='\\', names=['_projectid', '_teacherid', 'title', 'short_description', 'need_statement', 'essay', 'thankyou_note', 'impact_letter'])

In [2]:
essays.drop(['_projectid', '_teacherid', 'thankyou_note', 'impact_letter'], axis = 1, inplace = True, errors = 'ignore')

In [3]:
essays

,title,short_description,need_statement,essay
0,GOAL: Ordinary Students --> SUCCESSFUL SCIENTISTS,The first step towards academic achievement is...,"My students need Pencils, Crayons, Gluesticks,...",The first step towards academic achievement is...
1,OHMS Musician Chair Cart,The music students in our classes perform freq...,My students need a cart to help us manage our ...,The music students in our classes perform freq...
2,Can You Hear Me Now?,Find a way or make one. That's our class motto...,My students need headphones in order to engage...,Find a way or make one. That's our class motto...
3,Stand Up for Music!,"Despite high poverty, our district excels in t...",My students need music stands!,"Despite high poverty, our district excels in t..."
4,Gearing Up for Greatness - Full STEAM Ahead!,Our school serves Pre-K through Fifth grade st...,"My students need a STEM kits, books, and journ...",Our school serves Pre-K through Fifth grade st...
5,Projects Projecting into the 21st Century,Writing is a difficult process for many people...,My students need a document camera and project...,Writing is a difficult process for many people...
6,Projection to Capture Attention,I want my classroom to come alive like a Wii g...,My students need a DLP multi media projector.,As funding begins to disappear from our school...
7,Special Education Students Need Literary Resou...,Please help us buy books that we will love and...,"My students need 24 dictionaries, poetry antho...","My students are 17-19 years old. However, thei..."
8,Center Time!,Isn't it more fun to have hands on activities ...,"My students need magnifying glasses, and a var...",Isn't it more fun to have hands on activities ...
9,We Need Puzzles And Manipulatives!,Many of our class puzzles have been loved and ...,"My students need 6 floor puzzles, a set of bri...",Hello and thanks for checking out our proposal...


In [4]:
new = essays[:100]['short_description']

In [5]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [6]:
#nltk.download()
#Words that don't covey significant meaning
stopwords = nltk.corpus.stopwords.words('english')

#Stemming: breaking a word down into its root
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [7]:
#define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [8]:
#functions to iterate over the list of essays to create two vocabularies: 
# one stemmed and one only tokenized
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in new:#['short_description']:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [9]:
#create DataFrame with the stemmed vocabulary as the index and the tokenized words as the column. 
#The benefit of this is it provides an efficient way to look up a stem and return a full token
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 3614 items in vocab_frame


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(new)#['short_description']) #fit the vectorizer to synopses
print(tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
dist

(100, 6)


array([[ -2.22044605e-16,   6.92770774e-01,   8.42023687e-01, ...,
          9.27970216e-01,   7.49071967e-01,   1.00000000e+00],
       [  6.92770774e-01,   0.00000000e+00,   2.33217052e-01, ...,
          8.80717739e-01,   2.16804864e-01,   1.00000000e+00],
       [  8.42023687e-01,   2.33217052e-01,  -2.22044605e-16, ...,
          9.38665433e-01,   2.77396177e-01,   1.00000000e+00],
       ..., 
       [  9.27970216e-01,   8.80717739e-01,   9.38665433e-01, ...,
         -2.22044605e-16,   9.62233780e-01,   1.40819932e-02],
       [  7.49071967e-01,   2.16804864e-01,   2.77396177e-01, ...,
          9.62233780e-01,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          1.40819932e-02,   1.00000000e+00,   0.00000000e+00]])

In [11]:
from sklearn.cluster import KMeans

'''Number of clusters??'''
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

Wall time: 78.1 ms


In [12]:
pd.value_counts(pd.Series(clusters))# = new['cluster']#.value_counts()

0    24
3    22
1    20
2    19
4    15
dtype: int64

In [24]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in new.ix[i]:
        print('%s,' % title, end='')
    print() #add whitespace
    print() #add whitespace

Top terms per cluster:

Cluster 0 words: b'need', b'students', b'classroom', b'learning', b'read', b'school',

Cluster 0 titles:T,h,e, ,f,i,r,s,t, ,s,t,e,p, ,t,o,w,a,r,d,s, ,a,c,a,d,e,m,i,c, ,a,c,h,i,e,v,e,m,e,n,t, ,i,s, ,b,e,i,n,g, ,p,r,e,p,a,r,e,d,., ,H,e,l,p, ,m,y, ,s,t,u,d,e,n,t,s, ,f,i,n,d, ,t,h,e, ,p,r,e,p,a,r,a,t,i,o,n, ,t,h,e,y, ,n,e,e,d, ,a,l,r,e,a,d,y, ,t,h,e,r,e, ,i,n, ,t,h,e, ,c,l,a,s,s,r,o,o,m,., , ,M,y, ,s,t,u,d,e,n,t,s, ,n,e,e,d, ,p,e,n,c,i,l,s,,, ,c,r,a,y,o,n,s,,, ,g,l,u,e,s,t,i,c,k,s,,, ,s,c,i,s,s,o,r,s,,, ,a,n,d, ,f,o,l,d,e,r,s,., ,I, ,w,a,n,t, ,t,h,e, ,s,u,p,p,l,i,e,s, ,t,o, ,b,e, ,r,i,g,h,t, ,t,h,e,r,e, ,a,t, ,t,h,e,i,r, ,h,a,n,d,s, ,s,o, ,w,e,.,.,.,

Cluster 1 words: b'learning', b'students', b'classroom', b'school', b'read', b'need',

Cluster 1 titles:T,h,e, ,m,u,s,i,c, ,s,t,u,d,e,n,t,s, ,i,n, ,o,u,r, ,c,l,a,s,s,e,s, ,p,e,r,f,o,r,m, ,f,r,e,q,u,e,n,t,l,y, ,t,h,r,o,u,g,h,o,u,t, ,t,h,e, ,s,c,h,o,o,l, ,y,e,a,r,., ,T,h,i,s, ,m,e,a,n,s, ,m,o,v,i,n,g, ,a, ,l,o,t, ,o,f, ,

In [ ]:
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

In [ ]:
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

In [ ]:
from gensim import corpora, models, similarities 

#remove proper names
%time preprocess = [strip_proppers(doc) for doc in new]#['short_description']]

#tokenize
%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

#remove stop words
%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

In [ ]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary,update_every=5,chunksize=10000,passes=100)

lda.show_topics()

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

In [ ]:
topics_matrix